In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install fix_yahoo_finance

Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as MSE
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
yfin.pdr_override()

# Input Variables

In [4]:
Ticker = "MSFT"
Prediction_Date = "2010-07-01"
End_Learning = "2010-06-30"
Start_Learning = "2008-04-02"
Start_Prediction = "2010-06-30"
End_Prediction = "2010-07-03"

In [5]:
from pandas_datareader import data as pdr

df1 = pdr.get_data_yahoo(Ticker, 
                         start= Start_Learning,
                         end= End_Learning)

[*********************100%***********************]  1 of 1 completed


In [6]:
df1

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2008-04-01,28.830000,29.540001,28.629999,29.500000,21.899218,65796200
2008-04-02,29.570000,29.580000,29.000000,29.160000,21.646816,49499400
2008-04-03,29.000000,29.320000,28.799999,29.000000,21.528038,38961400
2008-04-04,29.129999,29.260000,28.740000,29.160000,21.646816,43860800
2008-04-07,29.549999,29.590000,29.030001,29.160000,21.646816,44428600
...,...,...,...,...,...,...
2010-06-23,25.780001,25.780001,25.219999,25.309999,19.688120,61466200
2010-06-24,25.459999,25.719999,24.930000,25.000000,19.446974,85243400
2010-06-25,25.049999,25.110001,24.309999,24.530001,19.081375,156256700


In [7]:
daily_prices1 = pd.concat([df1['Open'], df1['High'], df1['Low'],
                           df1['Adj Close'], df1['Volume'],df1['Close']], axis=1)

daily_prices1.columns = ['Open', 'High', 'Low',
                         'Adj Close','Volume','Close']

In [8]:
daily_prices1.head()

,Open,High,Low,Adj Close,Volume,Close
Date,,,,,,
2008-04-01,28.830000,29.540001,28.629999,21.899218,65796200,29.50
2008-04-02,29.570000,29.580000,29.000000,21.646816,49499400,29.16
2008-04-03,29.000000,29.320000,28.799999,21.528038,38961400,29.00
2008-04-04,29.129999,29.260000,28.740000,21.646816,43860800,29.16
2008-04-07,29.549999,29.590000,29.030001,21.646816,44428600,29.16


In [9]:
daily_prices1.tail()

,Open,High,Low,Adj Close,Volume,Close
Date,,,,,,
2010-06-23,25.780001,25.780001,25.219999,19.688120,61466200,25.309999
2010-06-24,25.459999,25.719999,24.930000,19.446974,85243400,25.000000
2010-06-25,25.049999,25.110001,24.309999,19.081375,156256700,24.530001
2010-06-28,24.510000,24.610001,24.120001,18.910240,73784800,24.309999
2010-06-29,24.129999,24.200001,23.110001,18.132368,119882100,23.309999


In [10]:
daily_prices1['NextDayClose'] = 0.001

for i in range (len(daily_prices1['Close'])):
    daily_prices1['NextDayClose'][i-1]=float(daily_prices1['Close'][i])

In [11]:
data1 = daily_prices1[daily_prices1.index != End_Learning]

In [12]:
data1.head()

,Open,High,Low,Adj Close,Volume,Close,NextDayClose
Date,,,,,,,
2008-04-01,28.830000,29.540001,28.629999,21.899218,65796200,29.50,29.16
2008-04-02,29.570000,29.580000,29.000000,21.646816,49499400,29.16,29.00
2008-04-03,29.000000,29.320000,28.799999,21.528038,38961400,29.00,29.16
2008-04-04,29.129999,29.260000,28.740000,21.646816,43860800,29.16,29.16
2008-04-07,29.549999,29.590000,29.030001,21.646816,44428600,29.16,28.75


In [13]:
data1.tail()

,Open,High,Low,Adj Close,Volume,Close,NextDayClose
Date,,,,,,,
2010-06-23,25.780001,25.780001,25.219999,19.688120,61466200,25.309999,25.000000
2010-06-24,25.459999,25.719999,24.930000,19.446974,85243400,25.000000,24.530001
2010-06-25,25.049999,25.110001,24.309999,19.081375,156256700,24.530001,24.309999
2010-06-28,24.510000,24.610001,24.120001,18.910240,73784800,24.309999,23.309999
2010-06-29,24.129999,24.200001,23.110001,18.132368,119882100,23.309999,29.500000


In [14]:
X = data1.loc[:, data1.columns != 'NextDayClose']
y = data1.loc[:, data1.columns == 'NextDayClose']

In [15]:
X.shape

(567, 6)

In [16]:
y.shape

(567, 1)

# Train and Build a Linear Regression Model

In [17]:
import statsmodels.api as sm
# Add a constant to the independent value
X1 = sm.add_constant(X)
# make regression model 
model = sm.OLS(y, X1)
# fit model and print results
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:           NextDayClose   R-squared:                       0.978
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                     4084.
Date:                Wed, 18 May 2022   Prob (F-statistic):               0.00
Time:                        23:24:41   Log-Likelihood:                -540.92
No. Observations:                 567   AIC:                             1096.
Df Residuals:                     560   BIC:                             1126.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3160      0.187      1.693      0.0

In [18]:
lr=LinearRegression()
#fit the linear model on train data
lr_model=lr.fit(X,y)  

# Model Evaluation

In [19]:
from sklearn.metrics import mean_squared_error,mean_absolute_error
#predict using the model
y_pred=lr.predict(X)  
#calculate R squared
print("R squared of linear regression model:", "{:.2%}".format(lr.score(X, y)))
#calculate rmse
rmse=np.sqrt(mean_squared_error(y,y_pred))     
print('RMSE of linear regression model:',rmse)

R squared of linear regression model: 97.77%
RMSE of linear regression model: 0.6281745399291477


# Real-time Prediction

In [20]:
df2 = pdr.get_data_yahoo(["MSFT"], start= Start_Prediction, end= End_Prediction)

[*********************100%***********************]  1 of 1 completed


In [21]:
df2

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-06-29,24.129999,24.200001,23.110001,23.309999,18.132364,119882100
2010-06-30,23.299999,23.680000,22.950001,23.010000,17.899004,81050500
2010-07-01,23.090000,23.320000,22.730000,23.160000,18.015676,92239400
2010-07-02,23.360001,23.480000,23.049999,23.270000,18.101250,62485100


In [22]:
daily_prices2 = pd.concat([df2['Open'], df2['High'], df2['Low'],
                           df2['Adj Close'], df2['Volume'],df2['Close']], axis=1)
daily_prices2.columns = ['Open', 'High', 'Low',
                         'Adj Close','Volume','Close']

In [23]:
daily_prices2.head()

,Open,High,Low,Adj Close,Volume,Close
Date,,,,,,
2010-06-29,24.129999,24.200001,23.110001,18.132364,119882100,23.309999
2010-06-30,23.299999,23.680000,22.950001,17.899004,81050500,23.010000
2010-07-01,23.090000,23.320000,22.730000,18.015676,92239400,23.160000
2010-07-02,23.360001,23.480000,23.049999,18.101250,62485100,23.270000


In [24]:
daily_prices2.tail()

,Open,High,Low,Adj Close,Volume,Close
Date,,,,,,
2010-06-29,24.129999,24.200001,23.110001,18.132364,119882100,23.309999
2010-06-30,23.299999,23.680000,22.950001,17.899004,81050500,23.010000
2010-07-01,23.090000,23.320000,22.730000,18.015676,92239400,23.160000
2010-07-02,23.360001,23.480000,23.049999,18.101250,62485100,23.270000


In [25]:
daily_prices2['NextDayClose'] = 0.001
for i in range (len(daily_prices2['Close'])):
    daily_prices2['NextDayClose'][i-1] = float(daily_prices2['Close'][i])

In [26]:
data2 = daily_prices2[daily_prices2.index == Start_Prediction]

In [27]:
data2

,Open,High,Low,Adj Close,Volume,Close,NextDayClose
Date,,,,,,,
2010-06-30,23.299999,23.68,22.950001,17.899004,81050500,23.01,23.16


In [28]:
TRUE = data2['NextDayClose'][0]

In [29]:
new_data = data2.drop(['NextDayClose'], axis=1)
new_data = np.array(new_data).reshape(1, -1)
new_pred=lr.predict(new_data)
result = float(new_pred[0])
print("\033[1m The estimated price for", Prediction_Date, "is: $",round(result,4))
print("\033[1m The stock was trading on", Prediction_Date, "is: $", round(TRUE,4))

 The estimated price for 2010-07-01 is: $ 23.2129
 The stock was trading on 2010-07-01 is: $ 23.16


In [30]:
data3 = daily_prices2.drop(['NextDayClose'], axis=1, inplace=True)
data3 = daily_prices2[daily_prices2.index == Prediction_Date]
data3

,Open,High,Low,Adj Close,Volume,Close
Date,,,,,,
2010-07-01,23.09,23.32,22.73,18.015676,92239400,23.16
